## Databricks Workflow Pause/Unpause Utility

### Overview  
This notebook enables safe and efficient pausing or unpausing of scheduled Databricks jobs based on a user-defined keyword. It is ideal for operational testing, deployment workflows, and temporary job management.

---

### Core Functionality  
The notebook accepts two parameters to control its behavior:

- **`KEYWORD`**: Filters jobs by name (case-insensitive substring match)  
- **`OPERATION`**: Specifies whether to `PAUSE` or `UNPAUSE` matching jobs  

It performs the following operations:
- Identifies jobs whose names contain the keyword and have a defined schedule
- Applies the selected operation only to jobs that are currently in the opposite state:
  - `PAUSE` only affects jobs that are currently unpaused
  - `UNPAUSE` only affects jobs that are currently paused
- Logs all updates and skips for full traceability

---

### Parameters

| Name        | Description                                | Default | Allowed Values      |
|-------------|--------------------------------------------|---------|---------------------|
| `KEYWORD`   | Required string to match in job names       | (empty) | Any string          |
| `OPERATION` | Action to apply to matching jobs            | PAUSE   | `PAUSE`, `UNPAUSE`  |

---

### Behavior Summary
- Only scheduled jobs matching the keyword are considered  
- Only jobs that need to be updated (based on current pause status) are affected  
- Detailed logging ensures transparency of operations

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import CronSchedule, PauseStatus
import logging

In [0]:
logger = logging.getLogger("pause_jobs")
logger.setLevel(logging.INFO)

In [0]:
KEYWORD = dbutils.widgets.get("KEYWORD").strip().lower()
OPERATION = dbutils.widgets.get("OPERATION").strip().upper()

assert KEYWORD, "KEYWORD cannot be empty — please provide a keyword to filter jobs."

logger.info(f"Filtering jobs using keyword: '{KEYWORD}'")
logger.info(f"Operation: {OPERATION}")

In [0]:
w = WorkspaceClient()

In [0]:
all_jobs = list(w.jobs.list())
filtered_jobs = [
    job for job in all_jobs
    if KEYWORD in job.settings.name.lower() and job.settings.schedule is not None
]

logger.info(f"Matched {len(filtered_jobs)} jobs with keyword '{KEYWORD}' and schedules.")

In [0]:
for job in filtered_jobs:
    job_name = job.settings.name
    schedule = job.settings.schedule
    current_status = schedule.pause_status.value

    # Determine target status
    target_status = PauseStatus.PAUSED if OPERATION == "PAUSE" else PauseStatus.UNPAUSED

    if current_status == target_status.value:
        logger.info(f"[SKIP] '{job_name}' already {current_status.lower()}.")
        continue

    logger.info(f"[{OPERATION}] '{job_name}' is currently {current_status.lower()}. Updating...")

    # Update schedule with new pause status
    new_schedule = CronSchedule(
        quartz_cron_expression=schedule.quartz_cron_expression,
        timezone_id=schedule.timezone_id,
        pause_status=target_status
    )

    updated_settings = job.settings
    updated_settings.schedule = new_schedule

    # Apply the update
    w.jobs.update(
        job_id=job.job_id,
        new_settings=updated_settings
    )

    logger.info(f"[SUCCESS] {OPERATION}D job: '{job_name}'")